# Making vars that compare between time periods

In [2]:
pip install awswrangler

  Using cached awswrangler-3.9.0-py3-none-any.whl.metadata (17 kB)
Using cached awswrangler-3.9.0-py3-none-any.whl (381 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
!{sys.executable} -m pip install PyAthena

  Using cached pyathena-3.9.0-py3-none-any.whl.metadata (6.3 kB)
Using cached pyathena-3.9.0-py3-none-any.whl (75 kB)


In [29]:
import awswrangler as wr
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
from pyathena import connect
import matplotlib.pyplot as plt
import seaborn as sns

# With target added

In [5]:
df = pd.read_csv('default_ABT1.csv')

In [6]:
df['DATE'] = pd.to_datetime(df['DATE'])

In [7]:
#sort values adecuado para todos los calculos hacia atrás!
df=df.sort_values(by=['ID_BRANCH', 'DATE'])

In [8]:
def calculate_rolling_means(df, columns, window):
    # Create an empty DataFrame to hold the results
    result_df = pd.DataFrame()

    # Loop through each column in the list of columns
    for column in columns:
        # Calculate the rolling mean for the current column
        rolling_mean = df.set_index('DATE').groupby('ID_BRANCH', sort=False)[column].rolling(window).mean().round(2).to_numpy()

        # Add the rolling mean to the result DataFrame
        result_df[f'roll_mean_{window}_{column}'] = rolling_mean

    # Concatenate the original DataFrame with the result DataFrame
    result_df = pd.concat([df, result_df], axis=1)

    return result_df


In [9]:
df = calculate_rolling_means(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], '30D')
df = calculate_rolling_means(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], '7D')

In [10]:
def calculate_rolling_max(df, columns, window):
    # Create an empty DataFrame to hold the results
    result_df = pd.DataFrame()

    # Loop through each column in the list of columns
    for column in columns:
        # Calculate the rolling mean for the current column
        rolling_mean = df.set_index('DATE').groupby('ID_BRANCH', sort=False)[column].rolling(window).max().to_numpy()

        # Add the rolling mean to the result DataFrame
        result_df[f'roll_max_{window}_{column}'] = rolling_mean

    # Concatenate the original DataFrame with the result DataFrame
    result_df = pd.concat([df, result_df], axis=1)

    return result_df

In [11]:
df = calculate_rolling_max(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], '7D')
df = calculate_rolling_max(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], '30D')

In [12]:

def calculate_lag(df, columns, window):
    # Create an empty DataFrame to hold the results
    result_df = pd.DataFrame()

    # Loop through each column in the list of columns
    for column in columns:
        # Calculate the rolling mean for the current column
        rolling_mean = df.set_index('DATE').groupby('ID_BRANCH', sort=False)[column].shift(window).to_numpy()

        # Add the rolling mean to the result DataFrame
        result_df[f'lag_{window}_{column}'] = rolling_mean

    # Concatenate the original DataFrame with the result DataFrame
    result_df = pd.concat([df, result_df], axis=1)

    return result_df

In [13]:
df = calculate_lag(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 1)

In [14]:
df.columns

Index(['DATE', 'ID_BRANCH', 'TX', 'AMOUNT', 'BALANCE', 'TX_check',
       'TOTAL_AMOUNT_check', 'TX_MO', 'TOTAL_AMOUNT_MO', 'DSO_DEPO_DAY',
       'INACTIVE_DAYS', 'DEFAULT', 'AGENCY', 'ID_GROUP_COMPANY', 'START_DATE',
       'day_of_week', 'TARGET', 'roll_mean_30D_AMOUNT',
       'roll_mean_30D_BALANCE', 'roll_mean_30D_TOTAL_AMOUNT_MO',
       'roll_mean_30D_TOTAL_AMOUNT_check', 'roll_mean_30D_TX',
       'roll_mean_30D_TX_MO', 'roll_mean_30D_TX_check',
       'roll_mean_30D_DEFAULT', 'roll_mean_7D_AMOUNT', 'roll_mean_7D_BALANCE',
       'roll_mean_7D_TOTAL_AMOUNT_MO', 'roll_mean_7D_TOTAL_AMOUNT_check',
       'roll_mean_7D_TX', 'roll_mean_7D_TX_MO', 'roll_mean_7D_TX_check',
       'roll_mean_7D_DEFAULT', 'roll_max_7D_AMOUNT', 'roll_max_7D_BALANCE',
       'roll_max_7D_TOTAL_AMOUNT_MO', 'roll_max_7D_TOTAL_AMOUNT_check',
       'roll_max_7D_TX', 'roll_max_7D_TX_MO', 'roll_max_7D_TX_check',
       'roll_max_7D_DEFAULT', 'roll_max_30D_AMOUNT', 'roll_max_30D_BALANCE',
       'roll_max_30D

In [15]:
df = calculate_lag(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 2)
df = calculate_lag(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 7)
df = calculate_lag(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 30)

In [16]:
def calculate_rolling_sum(df, columns, window):
    # Create an empty DataFrame to hold the results
    result_df = pd.DataFrame()

    # Loop through each column in the list of columns
    for column in columns:
        # Calculate the rolling mean for the current column
        rolling_mean = df.set_index('DATE').groupby('ID_BRANCH', sort=False)[column].rolling(window).sum().to_numpy()

        # Add the rolling mean to the result DataFrame
        result_df[f'roll_sum_{window}_{column}'] = rolling_mean

    # Concatenate the original DataFrame with the result DataFrame
    result_df = pd.concat([df, result_df], axis=1)

    return result_df

In [17]:
df = calculate_rolling_sum(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 7)
df = calculate_rolling_sum(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 30)
df = calculate_rolling_sum(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 60)

In [32]:
df.to_csv('default_ABT2.csv', index = False)

In [19]:
df.columns

Index(['DATE', 'ID_BRANCH', 'TX', 'AMOUNT', 'BALANCE', 'TX_check',
       'TOTAL_AMOUNT_check', 'TX_MO', 'TOTAL_AMOUNT_MO', 'DSO_DEPO_DAY',
       ...
       'roll_sum_30_TX_check', 'roll_sum_30_DEFAULT', 'roll_sum_60_AMOUNT',
       'roll_sum_60_BALANCE', 'roll_sum_60_TOTAL_AMOUNT_MO',
       'roll_sum_60_TOTAL_AMOUNT_check', 'roll_sum_60_TX', 'roll_sum_60_TX_MO',
       'roll_sum_60_TX_check', 'roll_sum_60_DEFAULT'],
      dtype='object', length=105)

# days from opening?

In [ ]:
'START_DATE', 'DATE'

In [26]:
df['DATE'] = pd.to_datetime(df['DATE'])
df['START_DATE'] = pd.to_datetime(df['START_DATE'])

In [27]:
df.dtypes

DATE                              datetime64[ns]
ID_BRANCH                                 object
TX                                       float64
AMOUNT                                   float64
BALANCE                                  float64
                                       ...      
roll_sum_60_TOTAL_AMOUNT_check           float64
roll_sum_60_TX                           float64
roll_sum_60_TX_MO                        float64
roll_sum_60_TX_check                     float64
roll_sum_60_DEFAULT                      float64
Length: 105, dtype: object

In [30]:
df['ANTIGUEDAD_d'] = (df['DATE'] - df['START_DATE']) / np.timedelta64(1, 'D')
#df['ANTIGUEDAD_w'] = (df['DATE'] - df['START_DATE']) / np.timedelta64(1, 'W')
#df['ANTIGUEDAD_m'] = (df['DATE'] - df['START_DATE']) / np.timedelta64(1, 'M')

In [31]:
df['DATE'] = df['DATE'].dt.strftime('%Y-%m-%d')
df['START_DATE'] = df['START_DATE'].dt.strftime('%Y-%m-%d')

In [ ]:
def calculate_pct_change(df, columns, windows):
    # Create an empty DataFrame to hold the results
    result_df = pd.DataFrame()

    # Loop through each column in the list of columns
    for column in columns:
        # Loop through each window in the list of windows
        for window in windows:
            # Calculate the rolling mean for the current column and window
            rolling_mean = df.set_index('DATE').groupby('ID_BRANCH', sort=False)[column].rolling(window).mean().round(2)

            # Calculate the percentage difference between the rolling mean columns
            pct_change = rolling_mean.pct_change(axis='columns', periods=len(windows)-1)

            # Add the percentage difference to the result DataFrame
            result_df[f'pct_change_{window}_{column}'] = pct_change

    # Concatenate the original DataFrame with the result DataFrame
    result_df = pd.concat([df, result_df], axis=1)

    return result_df


In [ ]:
merged_df = calculate_pct_change(merged_df, ['BALANCE', 'OTHER_COLUMN1', 'OTHER_COLUMN2'], [2, 7, 30])

In [ ]:
# Create a new column 'sales_diff' that represents the difference
# in sales from the previous period
df['sales_diff'] = df['sales'].diff()

# The first value in 'sales_diff' will be NaN, since there's no previous value
# You can fill this NaN with 0 or another appropriate value
df['sales_diff'].fillna(0, inplace=True)

In [ ]:
# Create a new column 'sales_pct_change' that represents the percentage change
# in sales from the previous period
df['sales_pct_change'] = df['sales'].pct_change() #compara entre filas
df.pct_change(axis='columns', periods=-1) #compara entre columnas
#Despite the name of this method, it calculates fractional change (also known as per unit change or relative change) and not percentage change. If you need the percentage change, multiply these values by 100.